In [ ]:
# import everything you need
from src import *
import numpy as np
import matplotlib.pyplot as plt

# set plot properties
plt.rcParams.update({'font.size': 25,
                     'font.family': 'Candara',
                     'text.usetex': True
                     })

# notebook code reloading
%load_ext autoreload
%autoreload 2

# Settings

In [ ]:
# for the lattice
Lx, Ly = 2,2
lat = hilbert.lattice(Lx, Ly)

# for the network
alpha = 1/4

# for ground state optimization
eta = 0.01

# for dynamics
steps = 1500
endtime = 1.5
breakdown = lambda t: -2. # an unstable quench

# links to process
links = [(0,2)]

# **Ground state**

This is how you can get the ground state.

In [ ]:
# declare
gs = groundstate.descent(lat, alpha, eta)

# run
gs.optimize(miniter=1000)

Let's use my ground state instead.

In [ ]:
# load it from the provided file
initial_state = np.array(np.loadtxt('../initial_state.txt'), dtype = complex)

# NOTE: if you want to use the ground state from the descent class instead, uncomment the following line:
#initial_state = gs.states[-1].copy()
initial_state

# **Line tracking algorithm**

Later in the analysis, we want to create continuous lines of eigenvalues, which come out of the code as sorted by magnitude. This is the function that we can use to create connected lines from the provided matrix specta.

In [ ]:
# written by ChatGPT
def track_eigenvalues_with_prediction(sorted_eigvals):
    n_eig, n_time = sorted_eigvals.shape
    tracked = np.zeros_like(sorted_eigvals)
    tracked[:, 0] = sorted_eigvals[:, 0]
    tracked[:, 1] = sorted_eigvals[:, 1]

    for i in range(2, n_time):
        # Predict next position using simple linear extrapolation
        prediction = 2 * tracked[:, i - 1] - tracked[:, i - 2]
        curr_vals = sorted_eigvals[:, i].copy()
        assigned = np.zeros(n_eig, dtype=bool)

        for j in range(n_eig):
            distances = np.abs(curr_vals - prediction[j])
            distances[assigned] = np.inf
            min_idx = np.argmin(distances)
            tracked[j, i] = curr_vals[min_idx]
            assigned[min_idx] = True

    return tracked

# <span style='color: lightblue'> **Calculations** </span>

We're calculating NQS dynamics with the explicit integrator in regularizaiton formulation, and the implicit integrator. We're also fitting the entire ED dynamics to RBM at each time using infidelity. We're comparing the spectrum of the $S$-marix for all of these cases. Everything is done for the breakdown case of $\Delta = -2$.

## NQS

In [ ]:
# declare dynamics
dyn_reg = dynamics.evolution(lat, alpha, initial_state.copy(), 
                             steps, endtime, perturbator=breakdown,
                             formulation='regularization')

dyn_imp = dynamics.evolution(lat, alpha, initial_state.copy(), 
                             int(steps/2), endtime, perturbator=breakdown,
                             formulation='geometric',
                             integrator='implicit_midpoint')

In [ ]:
# run them
dyn_reg.run()
dyn_imp.run()

# process correlations
dyn_reg.process_links(links)
dyn_imp.process_links(links)

# process spectra
dyn_reg.process_spectrum()
dyn_imp.process_spectrum()

## Infidelity fit

In [ ]:
# we need ED first
ed_m2 = exact.ED(lat, steps, endtime, perturbator=breakdown)
ed_m2.run()

This might run a bit slow, depending on your machine.

In [ ]:
# then we need to fit RBM to it

# declare the dynamics class (but we won't run it)
dyn_fit = dynamics.evolution(lat, alpha, initial_state.copy(), steps, endtime)

# fit to ED
losses, conv_steps = dyn_fit.fit_dynamics(ed_m2.states, links = links,
                    start_from_last = True, # this option gives us nice, continuous lines
                    criterion=1e-8 # set the precision of the fit
                    ) 

# <span style='color: lightblue'> **Smallest eigenvalues** </span>  time step dependence

We're calculating the smallest nonzero eigenvalue for each run, precisely up until very low time steps.

## Explicit

In [ ]:
# initialize some times and initial parameters
breakdown_index = np.argmin(dyn_reg.spectrum[2:])
print("cusp depth = ", np.min(dyn_reg.spectrum[2:]), " at index ", breakdown_index)
breakdown_time = dyn_reg.times[breakdown_index]
starting_index = breakdown_index-25 if (breakdown_index-25)>=0 else 0 # check that it doesn't spill over into negative times 
ti = dyn_reg.times[starting_index]
tf = 2*breakdown_time - ti if (2*breakdown_time - ti) > ti else dyn_reg.times[-1] # same for ending times
winit_exp = dyn_reg.states[starting_index].copy()

# runs list
regulated_dt = []

In [ ]:
# settings for the runs
iterations = 20
steps_to_run = 100

# runs
for _ in range(iterations):

    # declare
    nqs_reg = dynamics.evolution(lat, alpha, winit_exp, steps_to_run, tf, 
        perturbator = breakdown, start=ti)
    
    # flags
    print("\nRunning dt = ", nqs_reg.dt)
    print("initial time = ", ti, ", final time = ", tf, "\n")

    # run and process
    nqs_reg.run()
    nqs_reg.process_links(links)
    nqs_reg.process_spectrum()
    
    # append
    regulated_dt.append(nqs_reg)

    # update for the next loop iteration
    run_spectral_lines = track_eigenvalues_with_prediction(np.array(nqs_reg.spectrum))
    breakdown_index = np.argmin(nqs_reg.spectrum[2:])
    print("cusp depth = ", np.min(nqs_reg.spectrum[2:]), " at index ", breakdown_index)
    breakdown_time = nqs_reg.times[breakdown_index]
    starting_index = breakdown_index-25 if (breakdown_index-25)>=0 else 0 # check that it doesn't spill over into negative times (it fucks up the indexing)
    ti = nqs_reg.times[starting_index]
    tf = 2*breakdown_time - ti if (2*breakdown_time - ti) > ti else nqs_reg.times[-1] # same for ending times
    winit_exp = nqs_reg.states[starting_index].copy()

    print("----------------------------------")

In [ ]:
# get relevant data
reg_dt_spectra = [
    track_eigenvalues_with_prediction(np.array(s.spectrum)) for s in regulated_dt]
reg_spectral_minima = [np.min(sp[2:]) for sp in reg_dt_spectra]
reg_timesteps = [idt.dt for idt in regulated_dt]

## Implicit

In [ ]:
# initialize some times and initial parameters
breakdown_time = np.pi/(2*np.sqrt(48)) # believe it or not, this is the actual breakdown time
ti = ed_m2.times[201] # initial time
tf = 2*breakdown_time - ti
winit_imp = dyn_imp.states[101].copy()

# runs list
implicit_dt = []

In [ ]:
iterations = 20
steps_to_run = 100
starting_index = 40

for _ in range(iterations):
    # declare
    nqsrun_imp = dynamics.evolution(lat, alpha, winit_imp, steps_to_run, tf, 
        perturbator = breakdown, start=ti,
        formulation='geometric', integrator='implicit_midpoint')
    
    print("\nRunning dt = ", nqsrun_imp.dt)
    
    # run and process
    nqsrun_imp.run()
    nqsrun_imp.process_links(links)
    nqsrun_imp.process_spectrum()
    
    # append
    implicit_dt.append(nqsrun_imp)

    # update for the next loop iteration
    run_spectral_lines = track_eigenvalues_with_prediction(np.array(nqsrun_imp.spectrum))
    breakdown_index = np.argmin(run_spectral_lines[2])
    breakdown_time = nqsrun_imp.times[breakdown_index]
    starting_index = breakdown_index-25 if (breakdown_index-25)>=0 else 0
    ti = nqsrun_imp.times[starting_index]
    tf = 2*breakdown_time - ti if (2*breakdown_time - ti) > ti else nqsrun_imp.times[-1]
    winit_imp = nqsrun_imp.states[starting_index].copy()

    print("----------------------------------")

In [ ]:
# get the relevant data
implicit_dt_spectra = [
    track_eigenvalues_with_prediction(np.array(s.spectrum)) for s in implicit_dt]
imp_spectral_minima = [np.min(sp[2:]) for sp in implicit_dt_spectra]
imp_timesteps = [idt.dt for idt in implicit_dt]

## Infidelity fits

In [ ]:
# initialize some times and initial parameters
breakdown_time = np.pi/(2*np.sqrt(48)) # believe it or not, this is the actual breakdown time
ti = ed_m2.times[201] # initial time
tf = 2*breakdown_time - ti
winit_fit = dyn_fit.states[201].copy()
initial_psi = ed_m2.states[201].copy()

# runs list
fits_dt = []

This will probably run slow too.

In [ ]:
iterations = 20
steps_to_run = 100
starting_index = 40 # after run, index difference

for _ in range(iterations):

    # declare
    nqs_fit = dynamics.evolution(lat, alpha, winit_fit, steps_to_run, tf, 
        perturbator = breakdown, start=ti)
    
    # flags
    print("\nRunning dt = ", nqs_fit.dt)
    print("initial time = ", ti, ", final time = ", tf, "\n")

    # run ED
    ed_run = exact.ED(lat, steps_to_run, tf,
                      perturbator=breakdown, start = ti, initial=initial_psi)
    ed_run.run()

    # fit the fit
    h = nqs_fit.fit_dynamics(ed_run.states, criterion=1e-16, links=links)
    
    # append
    fits_dt.append(nqs_fit)

    # update for the next loop iteration
    run_spectral_lines = track_eigenvalues_with_prediction(np.array(nqs_fit.spectrum))
    breakdown_index = np.argmin(nqs_fit.spectrum[2:])
    print("cusp depth = ", np.min(nqs_fit.spectrum[2:]), " at index ", breakdown_index)
    breakdown_time = ed_run.times[breakdown_index]
    starting_index = breakdown_index-25 if (breakdown_index-25)>=0 else 0 # check that it doesn't spill over into negative times (it fucks up the indexing)
    ti = ed_run.times[starting_index]
    tf = 2*breakdown_time - ti if (2*breakdown_time - ti) > ti else ed_run.times[-1] # same for ending times
    winit_fit = nqs_fit.states[starting_index].copy()
    initial_psi = ed_run.states[starting_index].copy()
    
    print("----------------------------------")

In [ ]:
# get the data
fits_dt_spectra = [
    track_eigenvalues_with_prediction(np.array(s.spectrum)) for s in fits_dt]
#spectral_minima = [np.min(idt.spectrum[2:]) for idt in implicit_dt]
fit_spectral_minima = [np.min(sp[2:]) for sp in fits_dt_spectra]
fit_timesteps = [idt.dt for idt in fits_dt]

# <span style='color: lightgreen'> **Plots** </span>

## (a, b, c) Eigenvalues

In [ ]:
# get the lines from data
imp_spectral_lines = track_eigenvalues_with_prediction(np.array(dyn_imp.spectrum))
reg_spectral_lines = track_eigenvalues_with_prediction(np.array(dyn_reg.spectrum))
fit_spectral_lines = track_eigenvalues_with_prediction(np.array(dyn_fit.spectrum))

You might have to rearange which eigenvalues go on which graph. The line-connecting function doesn't sort them by anything. You might also have to re-run it, since the function is not perfect.

In [ ]:
# plt settings
fig, axes = plt.subplots(3, figsize=(12, 8), gridspec_kw={'height_ratios': [1, 1, 4]})
plt.subplots_adjust(hspace=0.1)
# bottom ticks
axes[0].tick_params(labelbottom=False)
axes[1].tick_params(labelbottom=False)

# visual settings
# from coolors.co: Off Red, Picton Blue, Sea Green, Licorice, either Silver: '#C1BDB3' or Timberwolf: '#D4D2D5'
cols = ['#F40000', '#00A6ED', '#09814A', '#291F1E', '#D4D2D5']
opacities = [0.75, 0.75, 0.75, 1]

# axes limits
for ax in axes:
    ax.set_xlim([0, 1.5])

# plots upper axis (largest eigenvalues)
axes[0].plot(dyn_imp.times, imp_spectral_lines[-3], lw = 4, alpha = opacities[3], color = cols[3], label='implicit')
axes[0].plot(dyn_reg.times, reg_spectral_lines[-3], lw = 4, alpha = opacities[1], color = cols[0], label = 'explicit')
axes[0].plot(dyn_fit.times, fit_spectral_lines[-3], lw = 3, ls = (0,(5,1)), color =cols[-2], alpha = 0.5, label = 'exact')


# plots middle axis (mid eigenvalues)
axes[1].plot(dyn_imp.times, imp_spectral_lines[-2], lw = 4, alpha = opacities[3], color = cols[3])
axes[1].plot(dyn_reg.times, reg_spectral_lines[-2], lw = 4, alpha = opacities[1], color = cols[0])
axes[1].plot(dyn_fit.times, fit_spectral_lines[-2], lw = 3, ls = (0,(5,1)), color =cols[-2], alpha = 0.5)

# plots lower axis finite eigenvalues
axes[2].plot(dyn_imp.times, imp_spectral_lines[-1], lw = 4, alpha = opacities[3], color = cols[3])
axes[2].plot(dyn_reg.times, reg_spectral_lines[-1], lw = 4, alpha = opacities[1], color = cols[0])
axes[2].plot(dyn_fit.times, fit_spectral_lines[-1], lw = 3, ls = (0,(5,1)), color =cols[-2], alpha = 0.5)

# plot lower axis zeros
for i in range(len(dyn_reg.spectrum[:2])):
    axes[2].plot(dyn_imp.times, imp_spectral_lines[i], lw = 4, alpha = opacities[3], color = cols[3])
    axes[2].plot(dyn_reg.times, reg_spectral_lines[i], lw = 4, alpha = opacities[1], color = cols[0])
    axes[2].plot(dyn_fit.times, fit_spectral_lines[i], lw = 3, ls = (0,(5,1)), color =cols[-2], alpha = 0.5)

# settings
axes[2].set_yscale('log')
axes[2].set_ylim([1e-17, 10])  

# some transition magic
axes[0].spines[['bottom']].set_visible(False)
axes[0].tick_params(axis='x', length=0)
axes[1].spines[['top']].set_visible(False)

# grids
axes[0].grid(True, which='both', linestyle='--', color='gray', alpha=0.7)
axes[1].grid(True, which='both', linestyle='--', color='gray', alpha=0.7)

# y limits
axes[1].set_ylim([0.4, 1.2])

# titles and labels
axes[2].set_xlabel("time $(1/J_0)$")
axes[0].set_title(r"correlation function $\left< S_i S_{i+1} \right>$", pad = 20)

# legend
axes[0].legend(ncols = 1, frameon = False,  bbox_to_anchor=(1, 0.5), loc='center left')

# save
plt.savefig("Smatrix_spectra.svg", format="svg", bbox_inches = "tight")

## Inset

In [ ]:
# reshape data
# fit
fit_timesteps_final =  [dyn_fit.dt] + fit_timesteps
fit_spectral_minima_final = [np.min(dyn_fit.spectrum[2:])] + fit_spectral_minima

# implicit
imp_timesteps_final = [dyn_imp.dt] + imp_timesteps
imp_spectral_minima_final = [np.min(dyn_imp.spectrum[2:])] + imp_spectral_minima

# regularization
reg_timesteps_final = [dyn_reg.dt] + reg_timesteps
reg_spectral_minima_final = [np.min(dyn_reg.spectrum[2:])] + reg_spectral_minima

You might want to cut out some data because the time steps tend to get super small. The data might not end up entirely identical in every run.

In [ ]:
fig, ax = plt.subplots(figsize=(6,4))

# plot the data
ax.plot(fit_timesteps_final[:19], fit_spectral_minima_final[:19], 
         color = 'gray', ls = (0,(5,1)), marker = 'o')
ax.plot(imp_timesteps_final, imp_spectral_minima_final, 
         color = '#291F1E', marker = 'o')
ax.plot(reg_timesteps_final[:15], reg_spectral_minima_final[:15], 
         color = '#F40000', marker = 'o')


# axis options
# plt.xlabel("time step")
# plt.ylabel("cusp depth")
ax.set_xscale("log")
ax.set_yscale("log")
ax.set_xlim([5e-10,3e-3])
ax.set_xticks([1e-9, 1e-7, 1e-5, 1e-3])
ax.yaxis.tick_right()
ax.yaxis.set_label_position("right")

# visual options
#plt.gca().spines[['right', 'top']].set_visible(False)
ax.grid(True, which='both', linestyle='--', color='gray', alpha=0.7)

plt.savefig("cusps_timesteps.svg", format="svg", bbox_inches="tight")

# <span style='color: pink'> **Saving** </span>

In [ ]:
# headers
head_eigenvalues = "time, [eigenvalue lines]"
head_inset = "time step, cusp depth"

## (a, b, c) Eigenvalue lines

In [ ]:
# explicit
data = np.c_[dyn_reg.times, reg_spectral_lines.T]

np.savetxt("data/eigenvalues/explicit_eigenvalues.txt", data, header = head_eigenvalues, delimiter=", ")

In [ ]:
# implicit
data = np.c_[dyn_imp.times, imp_spectral_lines.T]

np.savetxt("data/eigenvalues/implicit_eigenvalues.txt", data, header = head_eigenvalues, delimiter=", ")

In [ ]:
# fit
data = np.c_[dyn_fit.times, fit_spectral_lines.T]

np.savetxt("data/eigenvalues/fit_eigenvalues.txt", data, header = head_eigenvalues, delimiter=", ")

## (inset) Cusp depths

In [ ]:
# explicit
data = np.c_[reg_timesteps_final[:15], reg_spectral_minima_final[:15]]

np.savetxt("data/cusps/explicit_cusps.txt", data, header = head_inset, delimiter=", ")

In [ ]:
# implicit
data = np.c_[imp_timesteps_final, imp_spectral_minima_final]

np.savetxt("data/cusps/implicit_cusps.txt", data, header = head_inset, delimiter=", ")

In [ ]:
# fit
data = np.c_[fit_timesteps_final[:19], fit_spectral_minima_final[:19]]

np.savetxt("data/cusps/fit_cusps.txt", data, header = head_inset, delimiter=", ")